# Walking fly trajectories and Donut analysis

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## 1. Loading the data

In [ ]:
from object_detection_mini import load

In [ ]:
# todo: make the file smaller!!!

In [ ]:
# t_from=100
# t_to=300

In [ ]:
yml_config, df = load("flytrax20191119_111111.csv")

In [ ]:
yml_config

In [ ]:
df.head()

## 2. Plotting the data

In [ ]:
# "simple" way
plt.figure(figsize=(8,8))
plt.gca().axis('equal')
plt.plot(df.x_px, df.y_px, '.', markersize=1)

In [ ]:
df.t.max()

In [ ]:
from flyplot import plot_trajectory

In [ ]:
f, ax = plt.subplots(figsize=(8,8))
clrs = plot_trajectory(df.x_px, df.y_px, ax, tmax=600, markersize=10)
plt.colorbar(clrs, ax=ax)

### Indicate arena border, center and reward location

We will use matplotlib artist [Circle](https://matplotlib.org/api/_as_gen/matplotlib.patches.Circle.html)

In [ ]:
# cx, cy -- coordinates of arena center:
cx = yml_config['valid_region']['Circle']['center_x']
cy = yml_config['valid_region']['Circle']['center_y']
# arena_r -- arena radius
arena_r = yml_config['valid_region']['Circle']['radius']
print('Arena is a circle with center ({},{}) and radius {}'.format(cx, cy, arena_r))

In [ ]:
from matplotlib.patches import Circle

In [ ]:
ss_x = 660
ss_y = 565
ss_r = 50

In [ ]:
f, ax = plt.subplots(figsize=(13,13))

clrs = plot_trajectory(df.x_px, df.y_px, ax, tmin=t_from, tmax=t_to,markersize=10)
plt.colorbar(clrs, ax=ax)

# arena border
arena_artist = Circle((cx,cy),arena_r, ec='black', color='none', lw=2)
ax.add_artist(arena_artist)

# adjust ax limits so that the whole arena fits
ax.set_xlim(cx-arena_r-5, cx+arena_r+5)
ax.set_ylim(cy-arena_r-5, cy+arena_r+5)

# arena center
ax.scatter(cx,cy,marker='+', s=300, color='black', zorder=5)

# sweet spot
ss_artist = Circle((ss_x,ss_y),ss_r, ec='red', color='none', lw=2,zorder=5)
ax.add_artist(ss_artist)


## 3. Here comes the donut

### Assumptions: 
 - Arena is symmetric; the location preference in empty clean arena depends only on distance from the border.

So, if there is nothing special about the 'sweet spot', the flies should spend similar amount of time in any circle of radius $r_{ss}$ on the same distance from arena center. 

 - The temperature gradient is negligible inside the sweet spot

Now we will concider the area containing all the possible circles at that distance from arena center -- the donut!

Given the assumptions, for the ideal fly that walks infinitely in the ideal arena:
$$ \frac{S_{ss}}{S_o} = \frac{t_{ss}}{t_o} = \frac{N_{ss}}{N_o} $$
where $N_{area}$ is number of detections in area.

In [ ]:
# The donut parameters:
dss = np.sqrt((cx - ss_x)**2+(cy - ss_y)**2)

In [ ]:
dss

In [ ]:
r_inner = dss - ss_r
r_outer = dss + ss_r

In [ ]:
f, ax = plt.subplots(figsize=(13,13))

clrs = plot_trajectory(df.x_px, df.y_px, ax, tmin=t_from, tmax=t_to)
plt.colorbar(clrs, ax=ax)

# arena border
arena_artist = Circle((cx,cy),arena_r, ec='black', color='none', lw=2)
ax.add_artist(arena_artist)

# adjust ax limits so that the whole arena fits
ax.set_xlim(cx-arena_r-5, cx+arena_r+5)
ax.set_ylim(cy-arena_r-5, cy+arena_r+5)

# arena center
ax.scatter(cx,cy,marker='+', s=300, color='black', zorder=5)

# sweet spot
ss_artist = Circle((ss_x,ss_y),ss_r, ec='red', color='none', lw=2,zorder=5)
ax.add_artist(ss_artist)

# donut!
donut_inner_artist = Circle((cx,cy),r_inner, ec='brown', color='none', lw=2,zorder=5)
donut_outer_artist = Circle((cx,cy),r_outer, ec='brown', color='none', lw=2,zorder=5)
ax.add_artist(donut_inner_artist)
ax.add_artist(donut_outer_artist)


## 4. Counting detections 

We need the following values:
 - coordinates of arena center (`cx`, `cy`)
 - parameters of circle: radius, coordinates of center: (`ss_r`, `ss_x`, `ss_y`)

### 4.1 Inside the circle (sweet spot)

In [ ]:
df['inside_ss'] = (df.x_px - ss_x)**2 + (df.y_px - ss_y)**2 < ss_r**2

In [ ]:
df_in_ss = df[df.inside_ss]
df_outside_ss = df[df.inside_ss == False]
plt.plot(df_outside_ss.x_px, df_outside_ss.y_px, '.', markersize=1)
plt.plot(df_in_ss.x_px, df_in_ss.y_px, '.', markersize=1)
# plt.plot(df_in_ss.x_px, df_in_ss.y_px, '.')

In [ ]:
import seaborn as sns

In [ ]:
ax = sns.scatterplot(x="x_px", y="y_px", hue='inside_ss',markers=",", data=df,s=1, linewidth=0)


In [ ]:
n_all_detections = df.shape[0]

In [ ]:
n_ss_detections = df.inside_ss.sum()

In [ ]:
print("Detections in sweet spot: {} / {}".format(n_ss_detections, n_all_detections))

In [ ]:
df_in_ss.shape[0]

### 4.2 Inside the donut

In [ ]:
in_outer_circle = (df.x_px - cx)**2 + (df.y_px - cy)**2 < r_outer**2

In [ ]:
out_inner_circle = (df.x_px - cx)**2 + (df.y_px - cy)**2 > r_inner**2

In [ ]:
df_donut = df[in_outer_circle & out_inner_circle]

In [ ]:
plt.plot(df.x_px, df.y_px, '.', markersize=1)
plt.plot(df_donut.x_px, df_donut.y_px, '.', markersize=1)


In [ ]:
n_donut_detections = df_donut.shape[0]

In [ ]:
print("Detections in donut: {} / {}".format(n_donut_detections, n_all_detections))

## 5. Donut ratio

In [ ]:
donut_ratio_observed = n_ss_detections / n_donut_detections

In [ ]:
donut_ratio_observed

In [ ]:
s_ratio = ss_r / (4*dss)

In [ ]:
s_ratio

In [ ]:
s_ss=ss_r**2
s_inner = r_inner**2
s_outer = r_outer**2
s_donut = s_outer-s_inner
sratio = s_ss / s_donut

In [ ]:
sratio

In [ ]:
relative_donut_ratio = donut_ratio_observed / s_ratio

In [ ]:
relative_donut_ratio